In [92]:
%load_ext autoreload
%autoreload 2

from tbp.monty.frameworks.environments.ycb import YCBMeshDataset
import trimesh
import numpy as np
import torch 
import matplotlib.pyplot as plt
import seaborn as sns

#from nupic.research.frameworks.htm.temporal_memory import PairMemoryApicalTiebreak
from nupic.research.frameworks.columns import ApicalTiebreakPairMemory

#from scipy import sparse
#from scipy.sparse import csc_array, csr_array, coo_array

import pickle

import os, sys, inspect
sys.path.insert(1, os.path.expanduser("~/tbp/tbp.monty/projects/temporal_memory"))

from data_utils import *
from train import *

dataset = YCBMeshDataset(os.path.expanduser("~/tbp/data/habitat/objects/ycb"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%matplotlib qt 

fig, axes = plt.subplots(4, 5, figsize=(15, 15))
axes = axes.flatten()

for i in range(20):
    axes[i].hist(curvatures[i], bins='auto')

plt.show()

In [ ]:



def test_loop(object_id, train_active, train_objects, curve_hash_radius, curve_clusters, coord_hash_radius):
    curvatures, coordinates = get_test_data(object_id, curve_hash_radius=curve_hash_radius, curve_clusters=curve_clusters, coord_hash_radius=coord_hash_radius, num_samples=curve_clusters)

    testing_active_cells = []

    for curv, coord in zip(curvatures, coordinates):
        tm.compute(
            activeColumns=curv,
            basalInput=coord,
            learn=False
        )

        testing_active_cells.append(set(tm.getActiveCells().tolist()))

    """
    plot heatmap to describe overlap between training observations and testing observation
    """
    fig, axes = plt.subplots(ncols=len(train_objects), figsize=(10, 10))

    if not isinstance(axes, np.ndarray):
        axes = np.array([axes], dtype="O")

    mat = np.zeros((len(train_objects), len(testing_active_cells), len(train_active[0])))

    for t in range(len(train_objects)):
        for i in range(len(train_active[t])):
            for j in range(len(testing_active_cells)):
                #if i < j:
                if len(overlap(train_active[t][i], testing_active_cells[j])) > 20:
                    mat[t, j, i] = len(overlap(train_active[t][i], testing_active_cells[j]))

        sns.heatmap(mat[t, :], ax=axes[t], cbar=(t == (len(train_objects) - 1)))
        axes[t].set_xlabel("\nTrain Obj {0}".format(train_objects[t]))
        axes[t].set_title("{0}".format(int(mat[t, :].sum())))

        if t == 0:
            axes[t].set_ylabel("Inference Obj {0}\n".format(object_id))
    
    fig.suptitle("OBJECT {0}: INFERENCE. \n\nPREDICTED OBJECT: {1}".format(object_id, train_objects[mat.reshape(len(train_objects), -1).sum(axis=1).argmax()]))

    plt.show()

In [25]:
%matplotlib qt


"""
setup
"""

proximal_n = 2048
proximal_w = 30

basal_n = 2048
basal_w = 30

apical_n = 2048
apical_w = 30

tm = ApicalTiebreakPairMemory(
    columnCount=proximal_n,
    basalInputSize=basal_n,
    apicalInputSize=apical_n,
    cellsPerColumn=5, 
    activationThreshold=25,#10, 
    reducedBasalThreshold=25,#10,
    initialPermanence=0.51,
    connectedPermanence=0.5,
    minThreshold=25,#10,
    sampleSize=30,
    permanenceIncrement=0.1,
    permanenceDecrement=0.02,
    seed=42
)

"""
training
"""
train_objects = [5, 6, 7, 8, 9, 10]
#train_objects = list(range(20))

curve_hash_radius = 500
coord_hash_radius = 5
curve_clusters = 200

train_active = []

#for t in train_objects:
#    train_active.append(train_loop(t, curve_hash_radius=curve_hash_radius, curve_clusters=curve_clusters, coord_hash_radius=coord_hash_radius))

In [ ]:
"""
testing
"""

for t in train_objects:
    test_loop(t, train_active=train_active, train_objects=train_objects, curve_hash_radius=curve_hash_radius, curve_clusters=curve_clusters, coord_hash_radius=coord_hash_radius)

    print(t)

In [ ]:
# def train_loop(object_id, percent=1.0, num_iter=1):
#     curvatures, coordinates = get_data(object_id, percent)

#     training_active_cells = []

#     #for train_ind in range(0, len(curvatures), 2):
#     for train_ind in range(len(curvatures)):
#         if not len(curvatures[train_ind]):
#             pass

#         for _ in range(num_iter):
#             tm.compute(
#                 active_minicolumns=torch.from_numpy(curvatures[train_ind]),
#                 basal_input=torch.from_numpy(get_object_id(object_id, basal_n, basal_w)),
#                 apical_input=torch.from_numpy(coordinates[train_ind]),
#                 learn=True
#             )

#         training_active_cells.append(set(tm.get_active_cells().tolist()))

#     return training_active_cells

# def train_loop_og(object_id, percent=1.0, num_iter=1):
#     curvatures, coordinates = get_data(object_id, percent)

#     training_active_cells = []

#     #for train_ind in range(0, len(curvatures), 2):
#     for train_ind in range(len(curvatures)):
#         if not len(curvatures[train_ind]):
#             pass

#         for _ in range(num_iter):
#             tm_og.compute(
#                 activeColumns=curvatures[train_ind],
#                 basalInput=get_object_id(object_id, basal_n, basal_w),
#                 apicalInput=coordinates[train_ind],
#                 learn=True
#             )

#         training_active_cells.append(set(tm_og.getActiveCells().tolist()))

#     return training_active_cells


In [ ]:


# curvatures = proximal dendrites
# basal input = object ID
# apical input = location ID

# tm = PairMemoryApicalTiebreak(
#     num_minicolumns=proximal_n,
#     basal_input_size=basal_n,
#     apical_input_size=apical_n,
#     num_cells_per_minicolumn=5,
#     activation_threshold=10,
#     reduced_basal_threshold=10,
#     initial_permanence=0.5,
#     connected_permanence=0.5,
#     matching_threshold=10,
#     sample_size=30,
#     permanence_increment=0.1,
#     permanence_decrement=0.02,
#     basal_segment_incorrect_decrement=0.0,
#     seed=42
# )

'''
tm_og = ApicalTiebreakPairMemory(
    columnCount=proximal_n,
    basalInputSize=basal_n,
    apicalInputSize=apical_n,
    cellsPerColumn=5, # proximal_n * 5 = total cells
    activationThreshold=10, # 15 out of 30
    reducedBasalThreshold=10, # 15 out of 30
    initialPermanence=0.21, # --> initialPermanence=0.5 if not learning on higher order sequences
    connectedPermanence=0.5,
    minThreshold=8, # matching threshold == activation threshold == reduced basal threshold
    sampleSize=40, # at most w
    permanenceIncrement=0.1,
    permanenceDecrement=0.02,
    seed=42
)
'''

In [ ]:
"""
w = 105 out of n = 5000
proximal SDR input: (# points x 5000) (curvatures)
basal SDR input: (# points x 100) (object ID) <-- randomly generated (deterministic), 25 bits are ON of the 100 bits total
apical SDR input: (# points x 5000) (locations)

# points ~ 9000 - 16000

- reduce w and n
- w = 30, n = 2048
"""

In [ ]:
# basal connections: segments x (total # of possible synapses == total number of inputs bits == 100)

tm_og.basalConnections.matrix.toDense().shape

In [ ]:
# apical connections: segments x (total # of possible synapses == total number of input bits == 5000)

connections = tm_og.apicalConnections.matrix.toDense()

#x, y = connections.nonzero()

In [ ]:
(tm_og.apicalConnections.matrix.toDense()[0, :] != 0).sum()